# Классификация токсичных комментариев
Интернет-магазин запускает новый сервис. Теперь пользователи сами смогут редактировать описания товаров. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.


### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## План выполнения работы:
### <a href=#1>1. Подготовка</a>
-    #### <a href=#1_1> 1.1 Очистка и лемматизация текста</a>
-    #### <a href=#1_2> 1.2 Подготовка выборок</a>
-    #### <a href=#1_3> 1.3 Мешок слов</a>
-    #### <a href=#1_4> 1.4 TF-IDF</a>
-    #### <a href=#1_5> 1.5 word2vec</a>

### <a href=#2>2. Обучение</a>

### <a href=#3>3. Выводы</a>   

# <a id='1'> 1. Подготовка</a> 

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
import re 
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.metrics import f1_score

In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
toxic_comments.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


In [5]:
toxic_comments['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

## <a id='1_1'> 1.1 Очистка и лемматизация текста</a> 

In [6]:
#очистка от стопслов
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
lem=WordNetLemmatizer()
def preprocess_news(text):

    #очистка текста
    words = " ".join((re.sub(r'[^a-zA-Z ]', ' ', text)).split())
    #создание токенов
    words = [w for w in nltk.word_tokenize(words) if (w not in stopwords)]
    #лемматизация
    words = [lem.lemmatize(w.lower(), pos="v") for w in words]

    return ' '.join(words)


In [8]:
corpus = toxic_comments['text'].apply(preprocess_news)

In [9]:
corpus[1:3]

1    d aww he match background colour i seemingly s...
2    hey man i really try edit war it guy constantl...
Name: text, dtype: object

## <a id='1_2'> 1.2 Подготовка выборок</a>

In [10]:
target = toxic_comments.loc[:,'toxic']

Разобъём выборку на тестовые и тренировочные части

In [11]:
X_train, X_test, y_train, y_test = train_test_split(corpus, target,
                                                    test_size=0.2,
                                                    random_state=42)


In [12]:
print('Размер X_train: {:^10}'.format(X_train.shape[0]))
print('Размер y_train: {:^10} '.format(y_train.shape[0]))
print('Размер X_test:  {:^10}'.format(X_test.shape[0]))
print('Размер y_test:  {:^10} '.format(y_test.shape[0]))

Размер X_train:   127656  
Размер y_train:   127656   
Размер X_test:    31915   
Размер y_test:    31915    


## <a id='1_3'> 1.3 Мешок слов</a>

In [13]:
#мешок слов с стоп-словами
count_vect = CountVectorizer(stop_words=stopwords)

X_bof_train = count_vect.fit_transform(X_train)
X_bof_test = count_vect.transform(X_test)

In [14]:
print('Размер X_bof_train: {:^10} {:^10} '.format(X_bof_train.shape[0], X_bof_train.shape[1]))
print('Размер X_bof_test:  {:^10} {:^10} '.format(X_bof_test.shape[0], X_bof_test.shape[1]))

Размер X_bof_train:   127656     137023   
Размер X_bof_test:    31915      137023   


## <a id='1_4'> 1.4 TF-IDF</a>

In [15]:
#TF-IDF со стопсловами
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
X_tfidf_train = count_tf_idf.fit_transform(X_train) 
X_tfidf_test = count_tf_idf.transform(X_test) 

In [16]:
print('Размер X_tfidf_train: {:^10} {:^10} '.format(X_tfidf_train.shape[0], X_tfidf_train.shape[1]))
print('Размер X_tfidf_test:  {:^10} {:^10} '.format(X_tfidf_test.shape[0], X_tfidf_test.shape[1]))

Размер X_tfidf_train:   127656     137023   
Размер X_tfidf_test:    31915      137023   


## <a id='1_5'> 1.5 word2vec</a>

In [17]:
w2v_model = Word2Vec(size=300, min_count=1, window = 10, alpha=0.03)
w2v_model.build_vocab(X_train)

In [18]:
w2v_model.train(X_train, total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)
w2v = dict(zip(w2v_model.wv.index2word, w2v_model.wv.syn0))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [19]:
#найдём векторное представление каждого твита
def mean_transform(X):
    return np.array([np.mean([w2v[w] for w in words if w in w2v]
                             or [np.zeros(300)], axis=0)for words in X])


In [20]:
X_w2v_train = mean_transform(X_train)
X_w2v_test = mean_transform(X_test)

In [21]:
print('Размер X_w2v_train: {:^10} {:^10} '.format(X_w2v_train.shape[0], X_w2v_train.shape[1]))
print('Размер X_w2v_test:  {:^10} {:^10} '.format(X_w2v_test.shape[0], X_w2v_test.shape[1]))

Размер X_w2v_train:   127656      300     
Размер X_w2v_test:    31915       300     


# <a id='2'> 2. Обучение</a>

In [22]:
vec_features = {'bag_of_words': [X_bof_train, X_bof_test, y_train, y_test],
                'TF-IDF': [X_tfidf_train, X_tfidf_test, y_train, y_test],
                'word2vec': [X_w2v_train, X_w2v_test, y_train, y_test]}

In [23]:
#функция для предсказаний и вычисления метрики качества
def get_f1(model, X_test, y_test):
    y_pred = model.predict(X_test)
    score = f1_score(y_test, y_pred, average='binary')
    print('F1: {:.4f}'.format(score))
    return score

В качестве моделей возьмём `LogisticRegression`, `LinearSVC` и `LGBMRegressor`

In [24]:
log_reg_score=[] #сюда будем записывать метрику качества модели логистической регрессии
log_regres_model = LogisticRegression(C=2.5,
                                      class_weight='balanced', 
                                      random_state=42)
for key in vec_features.keys():
    log_regres_model.fit(vec_features[key][0], vec_features[key][2])
    score = get_f1(log_regres_model, vec_features[key][1], vec_features[key][3])
    log_reg_score.append(score)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


F1: 0.7328
F1: 0.7610
F1: 0.3504


In [25]:
svc_score = [] #для SVC соответственно
svc_model = LinearSVC(class_weight='balanced',
                      random_state=42)
for key in vec_features.keys():
    svc_model.fit(vec_features[key][0], vec_features[key][2])
    score = get_f1(svc_model, vec_features[key][1], vec_features[key][3])
    svc_score.append(score)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


F1: 0.7440
F1: 0.7634
F1: 0.3554


/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [26]:
kn_score = [] #KNeighbors
kn_model = KNeighborsClassifier(n_neighbors=10)
for key in vec_features.keys():
    kn_model.fit(vec_features[key][0], vec_features[key][2])
    score = get_f1(kn_model, vec_features[key][1], vec_features[key][3])
    kn_score.append(score)

F1: 0.3905
F1: 0.2298
F1: 0.2882


# <a id='3'> 3. Выводы</a>

In [27]:
def highlight_max(data, color='green'):
    #этим раскрасим максимальное значение метрики 
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [28]:
score_df = pd.DataFrame(data=log_reg_score, index=vec_features.keys(), columns=['LogisticRegression'])
score_df['LinearSVC'] = svc_score
score_df['KNeighbors'] = kn_score

In [29]:
score_df.style.apply(highlight_max, color='lightgreen', axis=None)

,LogisticRegression,LinearSVC,KNeighbors
bag_of_words,0.732796,0.744017,0.390458
TF-IDF,0.761038,0.763372,0.22981
word2vec,0.350408,0.355423,0.288165


**Вывод:** Из таблицы видно что лучшее значение метрики F1 *(0.7634)* до стигается при кодировании текстов TF-IDF на модели.

Возможно, word2vec дал был лучше результаты, если использовать преобученную модель на большем корпусе текстов. Скорее всего выборки недостоточно для обучения.